In [2]:
import pandas as pd

# Ruta a tu archivo Parquet
hydro = 'core_ferc1__yearly_hydroelectric_plants_sched406.parquet'

try:
    # Leer el archivo Parquet en un DataFrame de Pandas
    df = pd.read_parquet(hydro)

except FileNotFoundError:
    print(f"Error: El archivo '{hydro}' no se encontró.")
except Exception as e:
    print(f"Ocurrió un error al intentar leer el archivo Parquet: {e}")

In [9]:
import pandas as pd

# Ruta a tu archivo Parquet
ruta_archivo_parquet = 'core_ferc1__yearly_steam_plants_sched402.parquet'

try:
    # Leer el archivo Parquet en un DataFrame de Pandas
    df2 = pd.read_parquet(ruta_archivo_parquet)


except FileNotFoundError:
    print(f"Error: El archivo '{ruta_archivo_parquet}' no se encontró.")
except Exception as e:
    print(f"Ocurrió un error al intentar leer el archivo Parquet: {e}")

In [3]:
import pandas as pd

# Ruta al archivo
file_path = "fuel_consumed_units_number_Consumption of the fue....xlsx"

# Función auxiliar para dividir una hoja en dos tablas
def dividir_hoja(df, separador):
    idx = df[df.iloc[:, 0] == separador].index[0]
    tabla1 = df.iloc[:idx].dropna(how="all")
    tabla2 = df.iloc[idx:].dropna(how="all").reset_index(drop=True)
    tabla2.columns = tabla2.iloc[0]  # usar fila de encabezados reales
    tabla2 = tabla2.drop(0).reset_index(drop=True)
    return tabla1.reset_index(drop=True), tabla2

# Cargar y dividir hoja 1: Generación térmica
df_hoja1 = pd.read_excel(file_path, sheet_name="Generación térmica", header=None)
tabla_termica_campos, tabla_termica_categorias = dividir_hoja(df_hoja1, "Categoría FERC (Inglés)")

# Cargar y dividir hoja 2: Generación hidráulica
df_hoja2 = pd.read_excel(file_path, sheet_name="Generación hidráulica", header=None)
tabla_hidraulica_campos, tabla_hidraulica_categorias = dividir_hoja(df_hoja2, "Categoría FERC (Inglés)")

In [4]:
import pandas as pd
from IPython.display import display

# Mostrar todas las columnas y aumentar el ancho
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', None)

In [5]:
# Tabla 1: Campos – Generación Hidráulica
tabla_hidraulica_campos.columns = tabla_hidraulica_campos.iloc[0]
tabla_hidraulica_campos = tabla_hidraulica_campos.drop(0).reset_index(drop=True)
print("\n📌 Tabla 1: Campos – Generación Hidráulica")
display(tabla_hidraulica_campos)

# Tabla 2: Categorías – Generación Hidráulica
print("\n📌 Tabla 2: Categorías – Generación Hidráulica")
display(tabla_hidraulica_categorias)


📌 Tabla 1: Campos – Generación Hidráulica


,Field Name,Type,Descripción
0,asset_retirement_cost,number,Costo de retiro de activos (USD).
1,avg_num_employees,number,El número promedio de empleados asignados a cada planta.
2,capacity_mw,number,Capacidad total instalada (nominal) en megavatios.
3,capex_equipment,number,Costo de la planta: equipos (USD).
4,capex_facilities,number,"Costo de la planta: embalses, presas y vías navegables (USD)."
5,capex_land,number,Costo de la planta: terrenos y derechos de paso (USD).
6,capex_per_mw,number,Costo de la planta por megavatio instalado (nominal) de capacidad.
7,capex_roads,number,Costo de la planta: carreteras y puentes (USD).
8,capex_structures,number,Costo de la planta: estructuras y mejoras (USD).
9,capex_total,number,Costo total de la planta (USD).



📌 Tabla 2: Categorías – Generación Hidráulica


,Categoría FERC (Inglés),Traducción al Español,Descripción Técnica
0,hydro,Hidroeléctrica,Planta que genera electricidad a partir del movimiento del agua (sin distinción de tipo).
1,run_of_river,Hidroeléctrica de paso,"Generación sin gran embalse, el caudal del río fluye casi directamente por la turbina."
2,run_of_river_with_storage,Hidroeléctrica de paso con almacenamiento,"Similar a la de paso, pero con cierta capacidad de almacenamiento (embalses pequeños)."
3,storage,Hidroeléctrica de embalse / almacenamiento,Planta que utiliza un gran embalse para almacenar agua y generar energía cuando se requiere.


In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed
from IPython.display import display, HTML
from datetime import datetime
import re 
from ipywidgets import embed

# Configuración inicial
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

# Paleta de colores
COLOR_PALETTE = px.colors.qualitative.Plotly

## 1. Función para limpiar y preparar datos 
def prepare_data(df):
    """
    Prepara los datos asegurando formatos correctos y valores consistentes
    """
    df = df.copy()
    
    # Convertir año a datetime (manejo de diferentes formatos)
    if not pd.api.types.is_datetime64_any_dtype(df['report_year']):
        try:
            # Primero intentar convertir a entero y luego a datetime
            df['report_year'] = pd.to_datetime(df['report_year'].astype(int), format='%Y')
        except:
            # Si falla, intentar convertir directamente
            df['report_year'] = pd.to_datetime(df['report_year'])
    
    # Asegurar costos positivos
    cost_cols = [col for col in df.columns if 'capex_' in col or 'opex_' in col]
    for col in cost_cols:
        if col in df.columns: # Añadir esta comprobación para evitar KeyError si la columna no existe
            df[col] = df[col].abs()
    
    # Calcular totales si no existen
    if 'capex_total' not in df.columns:
        capex_cols = [col for col in df.columns if col.startswith('capex_') and col != 'capex_total' and col in df.columns]
        df['capex_total'] = df[capex_cols].sum(axis=1)
    
    if 'opex_total' not in df.columns:
        opex_cols = [col for col in df.columns if col.startswith('opex_') and col != 'opex_total' and col in df.columns]
        df['opex_total'] = df[opex_cols].sum(axis=1)
    
    # Calcular ratios con protección contra división por cero
    df['capex_per_mw'] = df.apply(
        lambda x: x['capex_total'] / x['capacity_mw'] if x['capacity_mw'] > 0 else 0,
        axis=1
    )
    
    df['opex_per_mwh'] = df.apply(
        lambda x: x['opex_total'] / x['net_generation_mwh'] if x['net_generation_mwh'] > 0 else 0,
        axis=1
    )
    
    # Asegurar que plant_type sea string y manejar valores nulos
    df['plant_type'] = df['plant_type'].astype(str)
    df['plant_type'] = df['plant_type'].replace('nan', 'No especificado')
    
    # Asegurar que plant_name_ferc1 sea string y manejar valores nulos
    if 'plant_name_ferc1' in df.columns:
        df['plant_name_ferc1'] = df['plant_name_ferc1'].astype(str)
        df['plant_name_ferc1'] = df['plant_name_ferc1'].replace('nan', 'No especificado')
        
        # Nuevo: Limpiar plant_name_ferc1 de posibles números entre paréntesis como "(1)"
        # Esto es crucial para que el nombre base coincida con el filtro de planta
        # Usamos una función para manejar NaN y aplicar la expresión regular
        def clean_plant_name(name):
            if pd.isna(name) or name == 'No especificado':
                return 'No especificado'
            return re.sub(r'\s*\(\d+\)\s*', '', str(name)).strip()

        df['plant_name_ferc1_clean'] = df['plant_name_ferc1'].apply(clean_plant_name)
        
        # Crear columna combinada para mostrar en el selector, usando el nombre limpio
        df['plant_name_with_type'] = df['plant_name_ferc1_clean'] + ' (' + df['plant_type'] + ')'
    else:
        # Si no hay 'plant_name_ferc1', crear una columna dummy para evitar errores
        df['plant_name_ferc1'] = 'No disponible'
        df['plant_name_ferc1_clean'] = 'No disponible'
        df['plant_name_with_type'] = 'No disponible'
        
    # NUEVA FUNCIONALIDAD: Crear columna de rangos de capacidad
    # Definir los bins de forma que siempre sean crecientes y el último abarque el valor máximo
    min_capacity = df['capacity_mw'].min()
    max_capacity = df['capacity_mw'].max()
    
    # Bins predefinidos
    predefined_bins = [0, 10, 50, 100, 250, 500, 1000, 2000]
    
    # Asegurarse de que el primer bin sea 0 o el mínimo valor si el mínimo es negativo (aunque capacity_mw es positivo)
    # y que el último bin abarque el máximo valor.
    bins = sorted(list(set(predefined_bins + [0, max_capacity + 1]))) # Usar set para eliminar duplicados y sorted para ordenar
    
    # Asegurarse de que el último bin es mayor que el penúltimo
    if len(bins) >= 2 and bins[-1] <= bins[-2]:
        bins[-1] = bins[-2] + 1 # Asegurar monotonicidad si el último bin se solapa

    # Generar las etiquetas dinámicamente basándose en los bins finales
    labels = []
    for i in range(len(bins) - 1):
        lower = bins[i]
        upper = bins[i+1] - 1 if i < len(bins) - 2 else bins[i+1] # Para el último bin, no restamos 1
        
        if i == len(bins) - 2 and upper == bins[-1]: # Si es el último rango y cubre hasta el final
             labels.append(f'>{lower} MW')
        elif i == 0 and lower == 0 and upper == 10: # Caso específico para el primer rango
            labels.append(f'{lower}-{upper} MW')
        elif i < len(bins) - 2 : # Rangos intermedios
             labels.append(f'{lower+1}-{upper} MW')
        else: # Si el último bin es un valor intermedio pero abarca hasta el final del rango
            labels.append(f'{lower+1}-{upper} MW')


    # Corrección para el caso donde el último bin cubre el máximo
    # Reconstrucción más sencilla y robusta de labels
    labels = []
    for i in range(len(bins) - 1):
        start = bins[i]
        end = bins[i+1]
        if i == len(bins) - 2: # Último intervalo
            labels.append(f'>{start} MW')
        elif start == 0: # Primer intervalo
            labels.append(f'{start}-{end} MW')
        else: # Intervalos intermedios
            labels.append(f'{start+1}-{end} MW')
            
    # Última revisión para asegurar que labels tenga el tamaño correcto
    if len(labels) != len(bins) - 1:
        # Esto no debería pasar con la lógica anterior, pero es una salvaguarda
        print("Advertencia: El número de etiquetas no coincide con el número de bins. Ajustando etiquetas.")
        labels = [f'Rango {i}' for i in range(len(bins)-1)] # Fallback genérico

    # Crear la nueva columna 'capacity_mw_range'
    try:
        df['capacity_mw_range'] = pd.cut(df['capacity_mw'], bins=bins, labels=labels, right=True, include_lowest=True)
    except ValueError as e:
        print(f"Error al crear rangos de capacidad: {e}")
        print(f"Valores de bins: {bins}")
        print(f"Valores de labels: {labels}")
        print(f"Máxima capacidad: {max_capacity}")
        # En caso de error, se puede asignar un valor por defecto.
        df['capacity_mw_range'] = 'Error de cálculo de rango' 
    
    df['capacity_mw_range'] = df['capacity_mw_range'].astype(str).replace('nan', 'No especificado')
    
    return df

## 2. Evolución de CAPEX, OPEX y Capacidad (MEJORADA)
def plot_cost_evolution(df, selected_years, selected_plant_types, selected_plants, selected_capacity_ranges):
    df_clean = prepare_data(df) # Asegurar que el dataframe que llega aquí ya esté limpio

    # Asegurar que los filtros sean listas y que el 'Todos' no esté presente en la comparación si es el caso
    years_to_filter = [y for y in selected_years if y != 'Todos'] if 'Todos' in selected_years else selected_years
    plant_types_to_filter = [pt for pt in selected_plant_types if pt != 'Todos'] if 'Todos' in selected_plant_types else selected_plant_types
    plants_to_filter = [p for p in selected_plants if p != 'Todos'] if 'Todos' in selected_plants else selected_plants
    capacity_ranges_to_filter = [cr for cr in selected_capacity_ranges if cr != 'Todos'] if 'Todos' in selected_capacity_ranges else selected_capacity_ranges

    selected_years_dt = [pd.to_datetime(str(year)) for year in years_to_filter]
    
    # Usar 'plant_name_ferc1_clean' para el filtrado si existe, de lo contrario 'plant_name_ferc1'
    plant_name_col_to_filter = 'plant_name_ferc1_clean' if 'plant_name_ferc1_clean' in df_clean.columns else 'plant_name_ferc1'

    filtered_df = df_clean[
        (df_clean['report_year'].isin(selected_years_dt) if years_to_filter else df_clean['report_year'].isin(df_clean['report_year'].unique())) & 
        (df_clean['plant_type'].isin(plant_types_to_filter) if plant_types_to_filter else df_clean['plant_type'].isin(df_clean['plant_type'].unique())) &
        (df_clean[plant_name_col_to_filter].isin(plants_to_filter) if plants_to_filter and plant_name_col_to_filter in df_clean.columns else pd.Series([True]*len(df_clean))) &
        (df_clean['capacity_mw_range'].isin(capacity_ranges_to_filter) if capacity_ranges_to_filter else pd.Series([True]*len(df_clean))) # Nuevo filtro por rango de capacidad
    ]

    if filtered_df.empty:
        display(HTML("""
        <div style="background-color:#fff3cd; padding:15px; border-radius:5px; margin-bottom:20px; border: 1px solid #ffeeba;">
            <p style="color:#856404; margin:0;">No hay datos disponibles para la combinación de filtros seleccionada para la Evolución de Costos.</p>
        </div>
        """))
        return # Salir de la función si no hay datos
        
    # Agrupar por año
    yearly_data = filtered_df.groupby('report_year').agg({
        'capex_total': 'sum',
        'opex_total': 'sum',
        'capacity_mw': 'sum',
        'net_generation_mwh': 'sum'
    }).reset_index()
    
    # Calcular ratios
    yearly_data['capex_per_mw'] = yearly_data['capex_total'] / yearly_data['capacity_mw'].replace(0, 1)
    yearly_data['opex_per_mwh'] = yearly_data['opex_total'] / yearly_data['net_generation_mwh'].replace(0, 1)
    
    # Crear figura
    fig = make_subplots(specs=[[{"secondary_y": True}]])
    
    # CAPEX (Barras)
    fig.add_trace(
        go.Bar(
            x=yearly_data['report_year'],
            y=yearly_data['capex_total'],
            name='CAPEX Total (USD)',
            marker_color=COLOR_PALETTE[0],
            opacity=0.7,
            hovertemplate='<b>CAPEX</b><br>Año: %{x|%Y}<br>Total: $%{y:,.0f}<extra></extra>'
        ),
        secondary_y=False
    )
    
    # OPEX (Barras)
    fig.add_trace(
        go.Bar(
            x=yearly_data['report_year'],
            y=yearly_data['opex_total'],
            name='OPEX Total (USD)',
            marker_color=COLOR_PALETTE[1],
            opacity=0.7,
            hovertemplate='<b>OPEX</b><br>Año: %{x|%Y}<br>Total: $%{y:,.0f}<extra></extra>'
        ),
        secondary_y=False
    )
    
    # Capacidad (Línea)
    fig.add_trace(
        go.Scatter(
            x=yearly_data['report_year'],
            y=yearly_data['capacity_mw'],
            name='Capacidad (MW)',
            line=dict(color=COLOR_PALETTE[2], width=3),
            mode='lines+markers',
            hovertemplate='<b>Capacidad</b><br>Año: %{x|%Y}<br>Total: %{y:,.0f} MW<extra></extra>'
        ),
        secondary_y=True
    )
    
    # Actualizar diseño
    fig.update_layout(
        title='Evolución de Costos y Capacidad',
        barmode='group',
        plot_bgcolor='white',
        hovermode='x unified',
        height=500,
        legend=dict(orientation="h", yanchor="bottom", y=1.1, x=0.5, xanchor="center"),  # Leyenda centrada
        margin=dict(t=120, b=80),  # Margen superior aumentado para la leyenda
        xaxis=dict(
            tickformat='%Y',  # Mostrar solo el año
            type='date'  # Asegurar que se trate como fecha
        )
    )
    
    # Configurar ejes
    fig.update_yaxes(title_text="Costos (USD)", secondary_y=False)
    fig.update_yaxes(title_text="Capacidad (MW)", secondary_y=True)
    fig.update_xaxes(title_text="Año")
    
    # Explicación introductoria
    explanation = """
    <div style="background-color:#f8f9fa; padding:15px; border-radius:5px; margin-bottom:20px">
        <h4>Análisis de Evolución de Costos</h4>
        <p>Este gráfico muestra la evolución temporal de los costos de capital (CAPEX) y operación (OPEX), 
        junto con la capacidad instalada total. Los valores de costos se muestran en USD (eje izquierdo) 
        mientras que la capacidad se muestra en MW (eje derecho).</p>
        <p><b>Nota:</b> Todos los valores de costos han sido convertidos a valores absolutos para evitar 
        interpretaciones erróneas con números negativos.</p>
    </div>
    """
    
    display(HTML(explanation))
    fig.show()

## 3. Composición de Costos Detallada (MEJORADA)
def plot_cost_composition(df, selected_years, selected_plant_types, selected_plants, selected_capacity_ranges):
    df_clean = prepare_data(df) # Asegurar que el dataframe que llega aquí ya esté limpio
    
    # Asegurar que los filtros sean listas y que el 'Todos' no esté presente en la comparación si es el caso
    years_to_filter = [y for y in selected_years if y != 'Todos'] if 'Todos' in selected_years else selected_years
    plant_types_to_filter = [pt for pt in selected_plant_types if pt != 'Todos'] if 'Todos' in selected_plant_types else selected_plant_types
    plants_to_filter = [p for p in selected_plants if p != 'Todos'] if 'Todos' in selected_plants else selected_plants
    capacity_ranges_to_filter = [cr for cr in selected_capacity_ranges if cr != 'Todos'] if 'Todos' in selected_capacity_ranges else selected_capacity_ranges

    selected_years_dt = [pd.to_datetime(str(year)) for year in years_to_filter]
    
    # Usar 'plant_name_ferc1_clean' para el filtrado si existe, de lo contrario 'plant_name_ferc1'
    plant_name_col_to_filter = 'plant_name_ferc1_clean' if 'plant_name_ferc1_clean' in df_clean.columns else 'plant_name_ferc1'

    filtered_df = df_clean[
        (df_clean['report_year'].isin(selected_years_dt) if years_to_filter else df_clean['report_year'].isin(df_clean['report_year'].unique())) & 
        (df_clean['plant_type'].isin(plant_types_to_filter) if plant_types_to_filter else df_clean['plant_type'].isin(df_clean['plant_type'].unique())) &
        (df_clean[plant_name_col_to_filter].isin(plants_to_filter) if plants_to_filter and plant_name_col_to_filter in df_clean.columns else pd.Series([True]*len(df_clean))) &
        (df_clean['capacity_mw_range'].isin(capacity_ranges_to_filter) if capacity_ranges_to_filter else pd.Series([True]*len(df_clean))) # Nuevo filtro por rango de capacidad
    ]
    
    if filtered_df.empty:
        display(HTML("""
        <div style="background-color:#fff3cd; padding:15px; border-radius:5px; margin-bottom:20px; border: 1px solid #ffeeba;">
            <p style="color:#856404; margin:0;">No hay datos disponibles para la combinación de filtros seleccionada para la Composición de Costos.</p>
        </div>
        """))
        return # Salir de la función si no hay datos

    # Componentes CAPEX (excluyendo totales)
    capex_cols = [col for col in df_clean.columns 
                  if col.startswith('capex_') 
                  and col not in ['capex_total', 'capex_per_mw']
                  and df_clean[col].sum() > 0 and col in filtered_df.columns] # Asegurar que la columna existe en el filtered_df
    
    # Componentes OPEX (excluyendo totales)
    opex_cols = [col for col in df_clean.columns 
                 if col.startswith('opex_') 
                 and col not in ['opex_total', 'opex_per_mwh']
                 and df_clean[col].sum() > 0 and col in filtered_df.columns] # Asegurar que la columna existe en el filtered_df
    
    # Crear figura con subplots
    fig = make_subplots(rows=2, cols=1, 
                        subplot_titles=("Composición CAPEX (USD)", "Composición OPEX (USD)"),
                        vertical_spacing=0.15)
    
    # ---- Gráfico CAPEX ----
    if capex_cols and not filtered_df.empty:
        capex_data = filtered_df.groupby('plant_type')[capex_cols].mean().reset_index()
        # Evitar división por cero en capex_pct
        sum_capex_by_plant_type = capex_data.set_index('plant_type').sum(axis=1).replace(0,1)
        capex_pct = capex_data.set_index('plant_type').div(sum_capex_by_plant_type, axis=0) * 100
        
        for i, col in enumerate(capex_cols):
            if not capex_data[col].empty:
                text_template = [f"${x:,.0f}<br>({p:.1f}%)" for x, p in 
                                 zip(capex_data[col], capex_pct[col])]
                
                fig.add_trace(
                    go.Bar(
                        x=capex_data['plant_type'],
                        y=capex_data[col],
                        name=col.replace('capex_', '').replace('_', ' ').title(),
                        marker_color=COLOR_PALETTE[i % len(COLOR_PALETTE)],
                        text=text_template,
                        textposition='auto',
                        hovertemplate='<b>%{fullData.name}</b><br>Tipo: %{x}<br>Valor: $%{y:,.0f}<br>Porcentaje: %{customdata:.1f}%<extra></extra>',
                        customdata=capex_pct[col],
                        textfont=dict(size=10),
                        showlegend=False
                    ),
                    row=1, col=1
                )
    
    # ---- Gráfico OPEX ----
    if opex_cols and not filtered_df.empty:
        opex_data = filtered_df.groupby('plant_type')[opex_cols].mean().reset_index()
        # Evitar división por cero en opex_pct
        sum_opex_by_plant_type = opex_data.set_index('plant_type').sum(axis=1).replace(0,1)
        opex_pct = opex_data.set_index('plant_type').div(sum_opex_by_plant_type, axis=0) * 100
        
        for i, col in enumerate(opex_cols):
            if not opex_data[col].empty:
                text_template = [f"${x:,.0f}<br>({p:.1f}%)" for x, p in 
                                 zip(opex_data[col], opex_pct[col])]
                
                fig.add_trace(
                    go.Bar(
                        x=opex_data['plant_type'],
                        y=opex_data[col],
                        name=col.replace('opex_', '').replace('_', ' ').title(),
                        marker_color=COLOR_PALETTE[(i + len(capex_cols)) % len(COLOR_PALETTE)],
                        text=text_template,
                        textposition='auto',
                        hovertemplate='<b>%{fullData.name}</b><br>Tipo: %{x}<br>Valor: $%{y:,.0f}<br>Porcentaje: %{customdata:.1f}%<extra></extra>',
                        customdata=opex_pct[col],
                        textfont=dict(size=10),
                        showlegend=False
                    ),
                    row=2, col=1
                )
    
    # Actualizar diseño
    fig.update_layout(
        title_text='Composición Detallada de Costos por Tipo de Planta',
        barmode='stack',
        plot_bgcolor='white',
        hovermode='x unified',
        height=900,
        margin=dict(t=100, b=80)
    )
    
    # Configurar ejes
    fig.update_yaxes(title_text="USD", row=1, col=1)
    fig.update_yaxes(title_text="USD", row=2, col=1)
    fig.update_xaxes(title_text="Tipo de Planta", row=2, col=1)
    
    # Explicación introductoria
    explanation = """
    <div style="background-color:#f8f9fa; padding:15px; border-radius:5px; margin-bottom:20px">
        <h4>Análisis de Composición de Costos</h4>
        <p>Esta visualización muestra el desglose detallado de los componentes que conforman los costos 
        de capital (CAPEX) y operación (OPEX) para cada tipo de planta seleccionada.</p>
        
        <p><b>Características:</b></p>
        <ul>
            <li><b>Gráficos separados:</b> CAPEX y OPEX mostrados en gráficos independientes para mayor claridad</li>
            <li><b>Valores absolutos:</b> Eje Y muestra valores en USD</li>
            <li><b>Porcentajes:</b> Cada barra muestra el valor en USD y el porcentaje que representa</li>
            <li><b>Interactividad:</b> Pase el cursor sobre las barras para ver detalles completos</li>
        </ul>
    </div>
    """
    
    display(HTML(explanation))
    fig.show()

## 4. Tabla Resumen con Explicación (MEJORADA)
def display_summary_table(df, selected_years, selected_plant_types, selected_plants, selected_capacity_ranges):
    df_clean = prepare_data(df) # Asegurar que el dataframe que llega aquí ya esté limpio
    
    # Asegurar que los filtros sean listas y que el 'Todos' no esté presente en la comparación si es el caso
    years_to_filter = [y for y in selected_years if y != 'Todos'] if 'Todos' in selected_years else selected_years
    plant_types_to_filter = [pt for pt in selected_plant_types if pt != 'Todos'] if 'Todos' in selected_plant_types else selected_plant_types
    plants_to_filter = [p for p in selected_plants if p != 'Todos'] if 'Todos' in selected_plants else selected_plants
    capacity_ranges_to_filter = [cr for cr in selected_capacity_ranges if cr != 'Todos'] if 'Todos' in selected_capacity_ranges else selected_capacity_ranges

    selected_years_dt = [pd.to_datetime(str(year)) for year in years_to_filter]
    
    # Usar 'plant_name_ferc1_clean' para el filtrado si existe, de lo contrario 'plant_name_ferc1'
    plant_name_col_to_filter = 'plant_name_ferc1_clean' if 'plant_name_ferc1_clean' in df_clean.columns else 'plant_name_ferc1'

    filtered_df = df_clean[
        (df_clean['report_year'].isin(selected_years_dt) if years_to_filter else df_clean['report_year'].isin(df_clean['report_year'].unique())) & 
        (df_clean['plant_type'].isin(plant_types_to_filter) if plant_types_to_filter else df_clean['plant_type'].isin(df_clean['plant_type'].unique())) &
        (df_clean[plant_name_col_to_filter].isin(plants_to_filter) if plants_to_filter and plant_name_col_to_filter in df_clean.columns else pd.Series([True]*len(df_clean))) &
        (df_clean['capacity_mw_range'].isin(capacity_ranges_to_filter) if capacity_ranges_to_filter else pd.Series([True]*len(df_clean))) # Nuevo filtro por rango de capacidad
    ]
    
    # MODIFICACIÓN: Mostrar mensaje si el DataFrame filtrado está vacío
    if filtered_df.empty:
        display(HTML("""
        <div style="background-color:#fff3cd; padding:15px; border-radius:5px; margin-bottom:20px; border: 1px solid #ffeeba;">
            <p style="color:#856404; margin:0;">No hay datos disponibles para la combinación de filtros seleccionada para la Tabla Resumen.</p>
        </div>
        """))
        return # Salir de la función si no hay datos
        
    # Calcular métricas resumen
    summary = filtered_df.groupby('plant_type').agg({
        'capex_total': ['sum', 'mean'],
        'opex_total': ['sum', 'mean'],
        'capacity_mw': 'sum',
        'net_generation_mwh': 'sum'
    })
    
    # Calcular ratios con protección contra división por cero
    summary['capex_per_mw'] = summary[('capex_total', 'sum')] / summary[('capacity_mw', 'sum')].replace(0, 1)
    summary['opex_per_mwh'] = summary[('opex_total', 'sum')] / summary[('net_generation_mwh', 'sum')].replace(0, 1)
    
    # Formatear tabla
    summary.columns = [' '.join(col).strip() for col in summary.columns.values]
    summary = summary[[
        'capacity_mw sum', 
        'net_generation_mwh sum',
        'capex_total sum', 
        'capex_total mean',
        'capex_per_mw',
        'opex_total sum',
        'opex_total mean',
        'opex_per_mwh'
    ]]
    
    # Renombrar columnas
    summary.columns = [
        'Capacidad Total (MW)',
        'Generación Total (MWh)',
        'CAPEX Total (USD)',
        'CAPEX Promedio (USD)',
        'CAPEX por MW (USD/MW)',
        'OPEX Total (USD)',
        'OPEX Promedio (USD)',
        'OPEX por MWh (USD/MWh)'
    ]
    
    # Explicación introductoria
    explanation = """
    <div style="background-color:#f8f9fa; padding:15px; border-radius:5px; margin-bottom:20px">
        <h4>Tabla Resumen de Costos</h4>
        <p>Esta tabla presenta un resumen consolidado de los costos y métricas clave para los tipos de planta 
        y años seleccionados. Los valores incluyen:</p>
        <ul>
            <li><b>CAPEX Total/Promedio:</b> Inversión total en capital y promedio por planta</li>
            <li><b>CAPEX por MW:</b> Costo de capital por unidad de capacidad instalada</li>
            <li><b>OPEX Total/Promedio:</b> Costos operativos totales y promedio por planta</li>
            <li><b>OPEX por MWh:</b> Costo operativo por unidad de energía generada</li>
        </ul>
    </div>
    """
    
    display(HTML(explanation))
    
    # Mostrar tabla con formato mejorado
    styled_table = summary.style.format({
        'CAPEX Total (USD)': '${:,.0f}',
        'CAPEX Promedio (USD)': '${:,.0f}',
        'CAPEX por MW (USD/MW)': '${:,.0f}',
        'OPEX Total (USD)': '${:,.0f}',
        'OPEX Promedio (USD)': '${:,.0f}',
        'OPEX por MWh (USD/MWh)': '${:,.2f}',
        'Capacidad Total (MW)': '{:,.0f}',
        'Generación Total (MWh)': '{:,.0f}'
    }).background_gradient(cmap='Blues', subset=['CAPEX Total (USD)', 'OPEX Total (USD)'])
    
    # Resaltar métricas clave
    styled_table = styled_table.set_properties(
        subset=['CAPEX por MW (USD/MW)', 'OPEX por MWh (USD/MWh)'],
        **{'background-color': '#f0f8ff', 'font-weight': 'bold'}
    )
    
    display(styled_table.set_caption("Resumen Consolidado de Costos"))

## Interfaz de Control Mejorada (MEJORADA)
def create_advanced_dashboard(df):
    # Preparar datos
    df_clean = prepare_data(df)
    
    # Obtener opciones para los widgets
    # Estas son las variables "all_..." que contienen todas las opciones posibles antes de cualquier filtro
    all_years = sorted(df_clean['report_year'].dt.year.unique())
    all_plant_types = sorted(df_clean['plant_type'].unique())
    
    # Usar 'plant_name_ferc1_clean' para obtener la lista de nombres de planta base para el filtrado interno
    all_plant_names_base = sorted(df_clean['plant_name_ferc1_clean'].unique()) if 'plant_name_ferc1_clean' in df_clean.columns else sorted(df_clean['plant_name_ferc1'].unique())
    
    # Generar 'all_plant_names_with_type' para el selector, que incluye el tipo y es el que se muestra al usuario
    all_plant_names_with_type = sorted(df_clean['plant_name_with_type'].unique()) if 'plant_name_with_type' in df_clean.columns else ['No disponible']

    # NUEVA FUNCIONALIDAD: Obtener todos los rangos de capacidad únicos
    all_capacity_ranges = sorted(df_clean['capacity_mw_range'].unique())
    # Filtra 'Error de cálculo de rango' de las opciones que se mostrarán en el selector inicial.
    all_capacity_ranges = [opt for opt in all_capacity_ranges if opt != 'Error de cálculo de rango']


    # Widgets con opción "Todos"
    year_selector = widgets.SelectMultiple(
        options=['Todos'] + all_years,
        value=['Todos'],
        description='Años:',
        disabled=False,
        style={'description_width': 'initial'},
        layout={'width': '300px'}
    )
    
    plant_type_selector = widgets.SelectMultiple(
        options=['Todos'] + all_plant_types,
        value=['Todos'],
        description='Tipos de Planta:',
        disabled=False,
        style={'description_width': 'initial'},
        layout={'width': '300px'}
    )
    
    # Nuevo: Widget de texto para el buscador de nombres de planta
    plant_name_search_bar = widgets.Text(
        value='',
        placeholder='Buscar nombre de planta...',
        description='Buscar:',
        disabled=False,
        style={'description_width': 'initial'},
        layout={'width': '300px'}
    )

    plant_name_selector = widgets.SelectMultiple(
        options=['Todos'] + all_plant_names_with_type, # Opciones iniciales completas
        value=['Todos'],
        description='Nombre de Planta:',
        disabled=False,
        style={'description_width': 'initial'},
        layout={'width': '300px'}
    )
    
    # NUEVO WIDGET: Selector para rangos de capacidad
    capacity_range_selector = widgets.SelectMultiple(
        options=['Todos'] + all_capacity_ranges,
        value=['Todos'],
        description='Rangos de Capacidad (MW):',
        disabled=False,
        style={'description_width': 'initial'},
        layout={'width': '300px'}
    )

    analysis_type = widgets.Dropdown(
        options=[
            ('Evolución de Costos', 'evolution'),
            ('Composición de Costos', 'composition'),
            ('Tabla Resumen', 'summary')
        ],
        value='evolution',
        description='Tipo de Análisis:',
        style={'description_width': 'initial'}
    )
    
    # Función para manejar la selección "Todos"
    def get_selected_options(selected, all_options):
        # Esta función ahora maneja el caso de 'No disponible' que puede venir del selector
        if 'Todos' in selected or len(selected) == 0:
            # Filtrar 'Error de cálculo de rango' si está presente en all_options
            return [opt for opt in all_options if opt not in ['No disponible', 'Error de cálculo de rango', 'nan']] 
        return [opt for opt in selected if opt not in ['No disponible', 'Error de cálculo de rango', 'nan']]


    # Función para extraer solo el nombre base de la planta (sin el tipo entre paréntesis y sin números como (1))
    def extract_plant_name_for_filter(plant_full_name_from_selector):
        if pd.isna(plant_full_name_from_selector) or plant_full_name_from_selector == 'No disponible':
            return 'No disponible' # Asegura que 'No disponible' no cause problemas
        
        # Eliminar el tipo de planta entre paréntesis (ej. ' (Nuclear)')
        name_without_type = plant_full_name_from_selector.split('(')[0].strip()
        
        # Eliminar números entre paréntesis si están en el nombre base (ej. 'Palo Verde (1)')
        name_cleaned = re.sub(r'\s*\(\d+\)\s*', '', name_without_type).strip()
        
        return name_cleaned
    
    # Función para actualizar las opciones de los selectores dependientes (años, tipos de planta y rangos de capacidad)
    def update_dependent_selectors(change):
        selected_years_filter = get_selected_options(year_selector.value, all_years)
        selected_plant_types_filter = get_selected_options(plant_type_selector.value, all_plant_types)
        
        # No usamos selected_capacity_ranges_filter aquí para determinar las opciones de los otros selectores,
        # ya que update_dependent_selectors debe recalcular TODAS las opciones para los SELECTORES,
        # y el filtro de rango de capacidad ya afectará a las opciones de las plantas.
        # Las opciones de rango de capacidad se actualizan en base a los filtros de año y tipo de planta, no al revés.

        # Filtrar el DataFrame según los años seleccionados (para plant_type_selector)
        selected_years_dt_filter = [pd.to_datetime(str(year)) for year in selected_years_filter]
        
        filtered_for_dependencies_df = df_clean[df_clean['report_year'].isin(selected_years_dt_filter)] if selected_years_dt_filter else df_clean
        
        # Actualizar opciones de plant_type_selector
        current_plant_type_options = sorted(filtered_for_dependencies_df['plant_type'].unique())
        new_plant_type_options = ['Todos'] + [opt for opt in current_plant_type_options if opt != 'nan'] # Filtrar 'nan'
        current_selection_types = [pt for pt in plant_type_selector.value if pt in new_plant_type_options]
        plant_type_selector.options = new_plant_type_options
        plant_type_selector.value = current_selection_types if current_selection_types else ['Todos']

        # Actualizar opciones de capacity_range_selector
        # Este filtro debe depender del año y tipo de planta
        current_capacity_range_options = sorted(filtered_for_dependencies_df['capacity_mw_range'].unique())
        # Filtra 'Error de cálculo de rango' y 'nan' de las opciones presentadas al usuario.
        current_capacity_range_options = [opt for opt in current_capacity_range_options if opt not in ['Error de cálculo de rango', 'nan']]
        new_capacity_range_options = ['Todos'] + current_capacity_range_options
        current_selection_ranges = [cr for cr in capacity_range_selector.value if cr in new_capacity_range_options]
        capacity_range_selector.options = new_capacity_range_options
        capacity_range_selector.value = current_selection_ranges if current_selection_ranges else ['Todos']
        
        # Vuelve a llamar a update_plant_name_options para refrescar las opciones de planta
        update_plant_name_options(None)


    # Nueva función para actualizar las opciones del selector de nombre de planta
    def update_plant_name_options(change):
        search_term = plant_name_search_bar.value.lower()
        
        # Aplicar el filtro de años y tipos de planta para las opciones de nombre de planta
        selected_years_filter = get_selected_options(year_selector.value, all_years)
        selected_plant_types_filter = get_selected_options(plant_type_selector.value, all_plant_types)
        selected_capacity_ranges_filter = get_selected_options(capacity_range_selector.value, all_capacity_ranges) # Obtener los rangos de capacidad seleccionados

        selected_years_dt_filter = [pd.to_datetime(str(year)) for year in selected_years_filter]

        filtered_df_for_plants = df_clean[
            (df_clean['report_year'].isin(selected_years_dt_filter)) & 
            (df_clean['plant_type'].isin(selected_plant_types_filter)) &
            (df_clean['capacity_mw_range'].isin(selected_capacity_ranges_filter)) # Filtro por rango de capacidad
        ]

        if 'plant_name_with_type' in filtered_df_for_plants.columns:
            # Obtener todas las opciones de nombre de planta relevantes para los filtros actuales
            current_relevant_plant_names = sorted(filtered_df_for_plants['plant_name_with_type'].unique())
            
            # Filtrar por el término de búsqueda
            filtered_by_search_plant_names = [
                pn for pn in current_relevant_plant_names
                if search_term in pn.lower()
            ]
            
            # Las nuevas opciones deben ser 'Todos' + las que cumplen la búsqueda
            new_plant_name_options = ['Todos'] + filtered_by_search_plant_names
            
            # Mantener la selección actual si sigue siendo válida
            current_selection_plants = [p for p in plant_name_selector.value if p in new_plant_name_options]
            plant_name_selector.options = new_plant_name_options
            plant_name_selector.value = current_selection_plants if current_selection_plants else ['Todos']
        else:
            plant_name_selector.options = ['No disponible']
            plant_name_selector.value = ['No disponible']
            
    # Asignar las funciones de observación a los widgets
    year_selector.observe(update_dependent_selectors, names='value')
    plant_type_selector.observe(update_dependent_selectors, names='value')
    capacity_range_selector.observe(update_dependent_selectors, names='value') # Nuevo observador
    plant_name_search_bar.observe(update_plant_name_options, names='value')

    # Ejecutar las actualizaciones iniciales para poblar los selectores al cargar el dashboard
    # update_dependent_selectors() llamará a update_plant_name_options()
    update_dependent_selectors(None) 

    # Función principal de actualización para los gráficos y tablas
    def update_analysis(analysis_type_val, selected_years_val, selected_plant_types_val, selected_plants_val, selected_capacity_ranges_val):
        
        # Obtener las opciones finales a usar, aplicando 'Todos'
        years_to_use = get_selected_options(selected_years_val, all_years)
        plant_types_to_use = get_selected_options(selected_plant_types_val, all_plant_types)
        capacity_ranges_to_use = get_selected_options(selected_capacity_ranges_val, all_capacity_ranges) # Obtener los rangos de capacidad seleccionados
        
        # Para las plantas, obtenemos los nombres del selector actual y los limpiamos para el filtro
        # Esto es crucial para que el filtro interno del DataFrame coincida con los nombres de la columna 'plant_name_ferc1_clean'
        selected_plants_from_selector_with_type = get_selected_options(selected_plants_val, plant_name_selector.options)
        
        # Limpiar los nombres de planta para el filtrado, usando la nueva función
        plants_to_use_for_filter = [extract_plant_name_for_filter(p) for p in selected_plants_from_selector_with_type]

        # Quitar duplicados por si acaso el extract_plant_name_for_filter genera los mismos nombres de diferentes variantes
        plants_to_use_for_filter = list(set(plants_to_use_for_filter))

        # Si después de limpiar y consolidar, 'Todos' sigue siendo una opción, usar la lista completa de nombres base
        if 'Todos' in plants_to_use_for_filter or not plants_to_use_for_filter:
            plants_to_use_for_filter = all_plant_names_base
        
        # Las funciones de ploteo ya contienen la lógica de filtrado del DataFrame,
        # así que pasamos los valores ya procesados.
        if analysis_type_val == 'evolution':
            plot_cost_evolution(df_clean, years_to_use, plant_types_to_use, plants_to_use_for_filter, capacity_ranges_to_use)
        elif analysis_type_val == 'composition':
            plot_cost_composition(df_clean, years_to_use, plant_types_to_use, plants_to_use_for_filter, capacity_ranges_to_use)
        elif analysis_type_val == 'summary':
            display_summary_table(df_clean, years_to_use, plant_types_to_use, plants_to_use_for_filter, capacity_ranges_to_use)
    
    # Interfaz interactiva
    ui = widgets.VBox([
        widgets.HBox([year_selector, plant_type_selector]),
        widgets.HBox([plant_name_search_bar, plant_name_selector]),
        widgets.HBox([capacity_range_selector]), # Agregamos el selector de rango de capacidad aquí
        analysis_type
    ])
    
    out = widgets.interactive_output(
        update_analysis,
        {
            'analysis_type_val': analysis_type,
            'selected_years_val': year_selector,
            'selected_plant_types_val': plant_type_selector,
            'selected_plants_val': plant_name_selector, 
            'selected_capacity_ranges_val': capacity_range_selector # Pasamos el valor del nuevo selector
        }
    )
    
    display(ui, out)

# Asumiendo que df ya está cargado en tu entorno, puedes ejecutar el dashboard
create_advanced_dashboard(df) 

Output()

In [7]:

# Tabla 3: Campos – Generación Térmica
tabla_termica_campos.columns = tabla_termica_campos.iloc[0]
tabla_termica_campos = tabla_termica_campos.drop(0).reset_index(drop=True)
print("📌 Tabla 3: Campos – Generación Térmica")
display(tabla_termica_campos)

# Tabla 4: Categorías – Generación Térmica
print("\n📌 Tabla 4: Categorías – Generación Térmica")
display(tabla_termica_categorias)




📌 Tabla 3: Campos – Generación Térmica


,Field Name,Type,Descripción
0,fuel_consumed_units,number,"Consumo del tipo de combustible en unidades físicas. Nota: esta es la cantidad total consumida tanto para electricidad como, en el caso de plantas de cogeneración, para la producción de vapor de proceso."
1,fuel_cost_per_mmbtu,number,Costo promedio del combustible por MMBTU de contenido de calor en USD nominales.
2,fuel_cost_per_unit_burned,number,Costo promedio del combustible consumido en el año del informe por unidad de combustible informada (USD).
3,fuel_cost_per_unit_delivered,number,Costo promedio del combustible entregado en el año del informe por unidad de combustible informada (USD).
4,fuel_mmbtu_per_unit,number,Contenido de calor del combustible en millones de BTUs por unidad física.
5,fuel_type_code_pudl,string,Código de tipo de combustible simplificado utilizado en PUDL.
6,fuel_units,string,Unidad de medida reportada para el combustible.
7,plant_name_ferc1,string,"Nombre de la planta, según lo informado a FERC. Esta es una cadena de texto libre, no garantizada para ser consistente entre referencias a la misma planta."
8,record_id,string,Identificador que indica el registro original de la fuente FERC Form 1. Formato: {table_name}{report_year}{report_prd}{respondent_id}{spplmnt_num}_{row_number}. Único dentro de las tablas de la base de datos de FERC Form 1 que no están mapeadas por fila.
9,report_year,integer,Año de cuatro dígitos en el que se informaron los datos.



📌 Tabla 4: Categorías – Generación Térmica


,Categoría FERC (Inglés),Traducción al Español,Descripción Técnica
0,wind,Eólica,Planta que genera electricidad a partir del viento mediante aerogeneradores.
1,combustion_turbine,Turbina de combustión,Planta que usa turbinas movidas por combustión de gas natural o petróleo.
2,combined_cycle,Ciclo combinado,Combina turbinas de gas y vapor para mejorar eficiencia térmica.
3,photovoltaic,Solar fotovoltaica,Usa paneles fotovoltaicos para convertir la luz solar directamente en electricidad.
4,solar_thermal,Solar termoeléctrica,Usa espejos para concentrar la luz solar y calentar un fluido que genera electricidad.
5,steam,Planta a vapor,"Generación térmica mediante calderas de vapor, típicamente a partir de carbón, gas o biomasa."
6,geothermal,Geotérmica,Utiliza calor del subsuelo para generar vapor y mover turbinas.
7,internal_combustion,Motor de combustión interna,"Motores similares a los de vehículos, pero usados para generación eléctrica."
8,nuclear,Nuclear,Usa la fisión nuclear para calentar agua y generar vapor para turbinas.


In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed
from IPython.display import display, HTML
from datetime import datetime
import re 
from ipywidgets import embed

# Configuración inicial
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

# Paleta de colores
COLOR_PALETTE = px.colors.qualitative.Plotly

## 1. Función para limpiar y preparar datos 
def prepare_data(df):
    """
    Prepara los datos asegurando formatos correctos y valores consistentes
    """
    df = df.copy()
    
    # Convertir año a datetime (manejo de diferentes formatos)
    if 'report_year' in df.columns and not pd.api.types.is_datetime64_any_dtype(df['report_year']):
        try:
            # Primero intentar convertir a entero y luego a datetime
            df['report_year'] = pd.to_datetime(df['report_year'].astype(int), format='%Y')
        except:
            # Si falla, intentar convertir directamente
            df['report_year'] = pd.to_datetime(df['report_year'])
    elif 'report_year' not in df.columns:
        print("Advertencia: 'report_year' no encontrada en el DataFrame. Algunas funcionalidades pueden verse afectadas.")
        df['report_year'] = pd.to_datetime('2000', format='%Y') # Default year if not found

    # Asegurar costos positivos
    cost_cols = [col for col in df.columns if 'capex_' in col or 'opex_' in col]
    for col in cost_cols:
        if col in df.columns: # Añadir esta comprobación para evitar KeyError si la columna no existe
            df[col] = df[col].abs()
    
    # Calcular totales si no existen
    if 'capex_total' not in df.columns:
        capex_cols = [col for col in df.columns if col.startswith('capex_') and col != 'capex_total' and col in df.columns]
        # Ensure that capex_cols are only numeric before summing
        capex_cols = [col for col in capex_cols if pd.api.types.is_numeric_dtype(df[col])]
        if capex_cols: # Only attempt sum if there are numeric capex columns
            df['capex_total'] = df[capex_cols].sum(axis=1)
        else:
            df['capex_total'] = 0 # Default to 0 if no capex columns

    if 'opex_total' not in df.columns:
        opex_cols = [col for col in df.columns if col.startswith('opex_') and col != 'opex_total' and col in df.columns]
        # Ensure that opex_cols are only numeric before summing
        opex_cols = [col for col in opex_cols if pd.api.types.is_numeric_dtype(df[col])]
        if opex_cols: # Only attempt sum if there are numeric opex columns
            df['opex_total'] = df[opex_cols].sum(axis=1)
        else:
            df['opex_total'] = 0 # Default to 0 if no opex columns
    
    # Calcular ratios con protección contra división por cero
    if 'capacity_mw' in df.columns:
        df['capex_per_mw'] = df.apply(
            lambda x: x['capex_total'] / x['capacity_mw'] if x['capacity_mw'] > 0 else 0,
            axis=1
        )
    else:
        df['capex_per_mw'] = 0
        print("Advertencia: 'capacity_mw' no encontrada. 'capex_per_mw' será 0.")

    if 'net_generation_mwh' in df.columns:
        df['opex_per_mwh'] = df.apply(
            lambda x: x['opex_total'] / x['net_generation_mwh'] if x['net_generation_mwh'] > 0 else 0,
            axis=1
        )
    else:
        df['opex_per_mwh'] = 0
        print("Advertencia: 'net_generation_mwh' no encontrada. 'opex_per_mwh' será 0.")
    
    # Asegurar que plant_type sea string y manejar valores nulos
    if 'plant_type' in df.columns:
        df['plant_type'] = df['plant_type'].astype(str)
        df['plant_type'] = df['plant_type'].replace('nan', 'No especificado')
    else:
        df['plant_type'] = 'No especificado'
        print("Advertencia: 'plant_type' no encontrada. Se usará 'No especificado'.")

    # Asegurar que plant_name_ferc1 sea string y manejar valores nulos
    if 'plant_name_ferc1' in df.columns:
        df['plant_name_ferc1'] = df['plant_name_ferc1'].astype(str)
        df['plant_name_ferc1'] = df['plant_name_ferc1'].replace('nan', 'No especificado')
        
        # Nuevo: Limpiar plant_name_ferc1 de posibles números entre paréntesis como "(1)"
        # Esto es crucial para que el nombre base coincida con el filtro de planta
        # Usamos una función para manejar NaN y aplicar la expresión regular
        def clean_plant_name(name):
            if pd.isna(name) or name == 'No especificado':
                return 'No especificado'
            return re.sub(r'\s*\(\d+\)\s*', '', str(name)).strip()

        df['plant_name_ferc1_clean'] = df['plant_name_ferc1'].apply(clean_plant_name)
        
        # Crear columna combinada para mostrar en el selector, usando el nombre limpio
        df['plant_name_with_type'] = df['plant_name_ferc1_clean'] + ' (' + df['plant_type'] + ')'
    else:
        # Si no hay 'plant_name_ferc1', crear una columna dummy para evitar errores
        df['plant_name_ferc1'] = 'No disponible'
        df['plant_name_ferc1_clean'] = 'No disponible'
        df['plant_name_with_type'] = 'No disponible'
        print("Advertencia: 'plant_name_ferc1' no encontrada. Se usará 'No disponible'.")
        
    # NUEVA FUNCIONALIDAD: Crear columna de rangos de capacidad
    if 'capacity_mw' in df.columns and df['capacity_mw'].sum() > 0:
        min_capacity = df['capacity_mw'].min()
        max_capacity = df['capacity_mw'].max()
        
        predefined_bins = [0, 10, 50, 100, 250, 500, 1000, 2000]
        
        bins = sorted(list(set(predefined_bins + [0, max_capacity + 1]))) 
        
        if len(bins) >= 2 and bins[-1] <= bins[-2]:
            bins[-1] = bins[-2] + 1 

        labels = []
        for i in range(len(bins) - 1):
            start = bins[i]
            end = bins[i+1]
            if i == len(bins) - 2: 
                labels.append(f'>{start} MW')
            elif start == 0: 
                labels.append(f'{start}-{end} MW')
            else: 
                labels.append(f'{start+1}-{end} MW')
                
        if len(labels) != len(bins) - 1:
            print("Advertencia: El número de etiquetas no coincide con el número de bins. Ajustando etiquetas.")
            labels = [f'Rango {i}' for i in range(len(bins)-1)] 

        try:
            df['capacity_mw_range'] = pd.cut(df['capacity_mw'], bins=bins, labels=labels, right=True, include_lowest=True)
        except ValueError as e:
            print(f"Error al crear rangos de capacidad: {e}")
            df['capacity_mw_range'] = 'Error de cálculo de rango' 
    else:
        df['capacity_mw_range'] = 'No especificado'
        print("Advertencia: 'capacity_mw' no encontrada o es cero. No se crearán rangos de capacidad.")
    
    df['capacity_mw_range'] = df['capacity_mw_range'].astype(str).replace('nan', 'No especificado')
    
    return df

## 2. Evolución de CAPEX, OPEX y Capacidad (MEJORADA)
def plot_cost_evolution(df, selected_years, selected_plant_types, selected_plants, selected_capacity_ranges):
    df_clean = prepare_data(df) 

    years_to_filter = [y for y in selected_years if y != 'Todos'] if 'Todos' in selected_years else selected_years
    plant_types_to_filter = [pt for pt in selected_plant_types if pt != 'Todos'] if 'Todos' in selected_plant_types else selected_plant_types
    plants_to_filter = [p for p in selected_plants if p != 'Todos'] if 'Todos' in selected_plants else selected_plants
    capacity_ranges_to_filter = [cr for cr in selected_capacity_ranges if cr != 'Todos'] if 'Todos' in selected_capacity_ranges else selected_capacity_ranges

    selected_years_dt = [pd.to_datetime(str(year)) for year in years_to_filter]
    
    plant_name_col_to_filter = 'plant_name_ferc1_clean' if 'plant_name_ferc1_clean' in df_clean.columns else 'plant_name_ferc1'

    filtered_df = df_clean[
        (df_clean['report_year'].isin(selected_years_dt) if years_to_filter else df_clean['report_year'].isin(df_clean['report_year'].unique())) & 
        (df_clean['plant_type'].isin(plant_types_to_filter) if plant_types_to_filter else df_clean['plant_type'].isin(df_clean['plant_type'].unique())) &
        (df_clean[plant_name_col_to_filter].isin(plants_to_filter) if plants_to_filter and plant_name_col_to_filter in df_clean.columns else pd.Series([True]*len(df_clean))) &
        (df_clean['capacity_mw_range'].isin(capacity_ranges_to_filter) if capacity_ranges_to_filter else pd.Series([True]*len(df_clean))) 
    ]

    if filtered_df.empty:
        display(HTML("""
        <div style="background-color:#fff3cd; padding:15px; border-radius:5px; margin-bottom:20px; border: 1px solid #ffeeba;">
            <p style="color:#856404; margin:0;">No hay datos disponibles para la combinación de filtros seleccionada para la Evolución de Costos.</p>
        </div>
        """))
        return 
        
    yearly_data = filtered_df.groupby('report_year').agg({
        'capex_total': 'sum',
        'opex_total': 'sum',
        'capacity_mw': 'sum',
        'net_generation_mwh': 'sum'
    }).reset_index()
    
    yearly_data['capex_per_mw'] = yearly_data['capex_total'] / yearly_data['capacity_mw'].replace(0, 1)
    yearly_data['opex_per_mwh'] = yearly_data['opex_total'] / yearly_data['net_generation_mwh'].replace(0, 1)
    
    fig = make_subplots(specs=[[{"secondary_y": True}]])
    
    fig.add_trace(
        go.Bar(
            x=yearly_data['report_year'],
            y=yearly_data['capex_total'],
            name='CAPEX Total (USD)',
            marker_color=COLOR_PALETTE[0],
            opacity=0.7,
            hovertemplate='<b>CAPEX</b><br>Año: %{x|%Y}<br>Total: $%{y:,.0f}<extra></extra>'
        ),
        secondary_y=False
    )
    
    fig.add_trace(
        go.Bar(
            x=yearly_data['report_year'],
            y=yearly_data['opex_total'],
            name='OPEX Total (USD)',
            marker_color=COLOR_PALETTE[1],
            opacity=0.7,
            hovertemplate='<b>OPEX</b><br>Año: %{x|%Y}<br>Total: $%{y:,.0f}<extra></extra>'
        ),
        secondary_y=False
    )
    
    fig.add_trace(
        go.Scatter(
            x=yearly_data['report_year'],
            y=yearly_data['capacity_mw'],
            name='Capacidad (MW)',
            line=dict(color=COLOR_PALETTE[2], width=3),
            mode='lines+markers',
            hovertemplate='<b>Capacidad</b><br>Año: %{x|%Y}<br>Total: %{y:,.0f} MW<extra></extra>'
        ),
        secondary_y=True
    )
    
    fig.update_layout(
        title='Evolución de Costos y Capacidad',
        barmode='group',
        plot_bgcolor='white',
        hovermode='x unified',
        height=500,
        legend=dict(orientation="h", yanchor="bottom", y=1.1, x=0.5, xanchor="center"),  
        margin=dict(t=120, b=80),  
        xaxis=dict(
            tickformat='%Y',  
            type='date'  
        )
    )
    
    fig.update_yaxes(title_text="Costos (USD)", secondary_y=False)
    fig.update_yaxes(title_text="Capacidad (MW)", secondary_y=True)
    fig.update_xaxes(title_text="Año")
    
    explanation = """
    <div style="background-color:#f8f9fa; padding:15px; border-radius:5px; margin-bottom:20px">
        <h4>Análisis de Evolución de Costos</h4>
        <p>Este gráfico muestra la evolución temporal de los costos de capital (CAPEX) y operación (OPEX), 
        junto con la capacidad instalada total. Los valores de costos se muestran en USD (eje izquierdo) 
        mientras que la capacidad se muestra en MW (eje derecho).</p>
        <p><b>Nota:</b> Todos los valores de costos han sido convertidos a valores absolutos para evitar 
        interpretaciones erróneas con números negativos.</p>
    </div>
    """
    
    display(HTML(explanation))
    fig.show()

## 3. Composición de Costos Detallada (MEJORADA)
def plot_cost_composition(df, selected_years, selected_plant_types, selected_plants, selected_capacity_ranges):
    df_clean = prepare_data(df) 
    
    years_to_filter = [y for y in selected_years if y != 'Todos'] if 'Todos' in selected_years else selected_years
    plant_types_to_filter = [pt for pt in selected_plant_types if pt != 'Todos'] if 'Todos' in selected_plant_types else selected_plant_types
    plants_to_filter = [p for p in selected_plants if p != 'Todos'] if 'Todos' in selected_plants else selected_plants
    capacity_ranges_to_filter = [cr for cr in selected_capacity_ranges if cr != 'Todos'] if 'Todos' in selected_capacity_ranges else selected_capacity_ranges

    selected_years_dt = [pd.to_datetime(str(year)) for year in years_to_filter]
    
    plant_name_col_to_filter = 'plant_name_ferc1_clean' if 'plant_name_ferc1_clean' in df_clean.columns else 'plant_name_ferc1'

    filtered_df = df_clean[
        (df_clean['report_year'].isin(selected_years_dt) if years_to_filter else df_clean['report_year'].isin(df_clean['report_year'].unique())) & 
        (df_clean['plant_type'].isin(plant_types_to_filter) if plant_types_to_filter else df_clean['plant_type'].isin(df_clean['plant_type'].unique())) &
        (df_clean[plant_name_col_to_filter].isin(plants_to_filter) if plants_to_filter and plant_name_col_to_filter in df_clean.columns else pd.Series([True]*len(df_clean))) &
        (df_clean['capacity_mw_range'].isin(capacity_ranges_to_filter) if capacity_ranges_to_filter else pd.Series([True]*len(df_clean))) 
    ]
    
    if filtered_df.empty:
        display(HTML("""
        <div style="background-color:#fff3cd; padding:15px; border-radius:5px; margin-bottom:20px; border: 1px solid #ffeeba;">
            <p style="color:#856404; margin:0;">No hay datos disponibles para la combinación de filtros seleccionada para la Composición de Costos.</p>
        </div>
        """))
        return 

    capex_cols = [col for col in df_clean.columns 
                    if col.startswith('capex_') 
                    and col not in ['capex_total', 'capex_per_mw']
                    and col in filtered_df.columns and pd.api.types.is_numeric_dtype(df_clean[col])] 
    
    opex_cols = [col for col in df_clean.columns 
                    if col.startswith('opex_') 
                    and col not in ['opex_total', 'opex_per_mwh']
                    and col in filtered_df.columns and pd.api.types.is_numeric_dtype(df_clean[col])] 
    
    fig = make_subplots(rows=2, cols=1, 
                        subplot_titles=("Composición CAPEX (USD)", "Composición OPEX (USD)"),
                        vertical_spacing=0.15)
    
    if capex_cols and not filtered_df.empty:
        capex_data = filtered_df.groupby('plant_type')[capex_cols].mean().reset_index()
        sum_capex_by_plant_type = capex_data.set_index('plant_type').sum(axis=1).replace(0,1)
        capex_pct = capex_data.set_index('plant_type').div(sum_capex_by_plant_type, axis=0) * 100
        
        for i, col in enumerate(capex_cols):
            if not capex_data[col].empty:
                text_template = [f"${x:,.0f}<br>({p:.1f}%)" for x, p in 
                                    zip(capex_data[col], capex_pct[col])]
                
                fig.add_trace(
                    go.Bar(
                        x=capex_data['plant_type'],
                        y=capex_data[col],
                        name=col.replace('capex_', '').replace('_', ' ').title(),
                        marker_color=COLOR_PALETTE[i % len(COLOR_PALETTE)],
                        text=text_template,
                        textposition='auto',
                        hovertemplate='<b>%{fullData.name}</b><br>Tipo: %{x}<br>Valor: $%{y:,.0f}<br>Porcentaje: %{customdata:.1f}%<extra></extra>',
                        customdata=capex_pct[col],
                        textfont=dict(size=10),
                        showlegend=False
                    ),
                    row=1, col=1
                )
    
    if opex_cols and not filtered_df.empty:
        opex_data = filtered_df.groupby('plant_type')[opex_cols].mean().reset_index()
        sum_opex_by_plant_type = opex_data.set_index('plant_type').sum(axis=1).replace(0,1)
        opex_pct = opex_data.set_index('plant_type').div(sum_opex_by_plant_type, axis=0) * 100
        
        for i, col in enumerate(opex_cols):
            if not opex_data[col].empty:
                text_template = [f"${x:,.0f}<br>({p:.1f}%)" for x, p in 
                                    zip(opex_data[col], opex_pct[col])]
                
                fig.add_trace(
                    go.Bar(
                        x=opex_data['plant_type'],
                        y=opex_data[col],
                        name=col.replace('opex_', '').replace('_', ' ').title(),
                        marker_color=COLOR_PALETTE[(i + len(capex_cols)) % len(COLOR_PALETTE)],
                        text=text_template,
                        textposition='auto',
                        hovertemplate='<b>%{fullData.name}</b><br>Tipo: %{x}<br>Valor: $%{y:,.0f}<br>Porcentaje: %{customdata:.1f}%<extra></extra>',
                        customdata=opex_pct[col],
                        textfont=dict(size=10),
                        showlegend=False
                    ),
                    row=2, col=1
                )
    
    fig.update_layout(
        title_text='Composición Detallada de Costos por Tipo de Planta',
        barmode='stack',
        plot_bgcolor='white',
        hovermode='x unified',
        height=900,
        margin=dict(t=100, b=80)
    )
    
    fig.update_yaxes(title_text="USD", row=1, col=1)
    fig.update_yaxes(title_text="USD", row=2, col=1)
    fig.update_xaxes(title_text="Tipo de Planta", row=2, col=1)
    
    explanation = """
    <div style="background-color:#f8f9fa; padding:15px; border-radius:5px; margin-bottom:20px">
        <h4>Análisis de Composición de Costos</h4>
        <p>Esta visualización muestra el desglose detallado de los componentes que conforman los costos 
        de capital (CAPEX) y operación (OPEX) para cada tipo de planta seleccionada.</p>
        
        <p><b>Características:</b></p>
        <ul>
            <li><b>Gráficos separados:</b> CAPEX y OPEX mostrados en gráficos independientes para mayor claridad</li>
            <li><b>Valores absolutos:</b> Eje Y muestra valores en USD</li>
            <li><b>Porcentajes:</b> Cada barra muestra el valor en USD y el porcentaje que representa</li>
            <li><b>Interactividad:</b> Pase el cursor sobre las barras para ver detalles completos</li>
        </ul>
    </div>
    """
    
    display(HTML(explanation))
    fig.show()

## 4. Tabla Resumen con Explicación (MEJORADA)
def display_summary_table(df, selected_years, selected_plant_types, selected_plants, selected_capacity_ranges):
    df_clean = prepare_data(df) 
    
    years_to_filter = [y for y in selected_years if y != 'Todos'] if 'Todos' in selected_years else selected_years
    plant_types_to_filter = [pt for pt in selected_plant_types if pt != 'Todos'] if 'Todos' in selected_plant_types else selected_plant_types
    plants_to_filter = [p for p in selected_plants if p != 'Todos'] if 'Todos' in selected_plants else selected_plants
    capacity_ranges_to_filter = [cr for cr in selected_capacity_ranges if cr != 'Todos'] if 'Todos' in selected_capacity_ranges else selected_capacity_ranges

    selected_years_dt = [pd.to_datetime(str(year)) for year in years_to_filter]
    
    plant_name_col_to_filter = 'plant_name_ferc1_clean' if 'plant_name_ferc1_clean' in df_clean.columns else 'plant_name_ferc1'

    filtered_df = df_clean[
        (df_clean['report_year'].isin(selected_years_dt) if years_to_filter else df_clean['report_year'].isin(df_clean['report_year'].unique())) & 
        (df_clean['plant_type'].isin(plant_types_to_filter) if plant_types_to_filter else df_clean['plant_type'].isin(df_clean['plant_type'].unique())) &
        (df_clean[plant_name_col_to_filter].isin(plants_to_filter) if plants_to_filter and plant_name_col_to_filter in df_clean.columns else pd.Series([True]*len(df_clean))) &
        (df_clean['capacity_mw_range'].isin(capacity_ranges_to_filter) if capacity_ranges_to_filter else pd.Series([True]*len(df_clean))) 
    ]
    
    if filtered_df.empty:
        display(HTML("""
        <div style="background-color:#fff3cd; padding:15px; border-radius:5px; margin-bottom:20px; border: 1px solid #ffeeba;">
            <p style="color:#856404; margin:0;">No hay datos disponibles para la combinación de filtros seleccionada para la Tabla Resumen.</p>
        </div>
        """))
        return 
        
    summary = filtered_df.groupby('plant_type').agg({
        'capex_total': ['sum', 'mean'],
        'opex_total': ['sum', 'mean'],
        'capacity_mw': 'sum',
        'net_generation_mwh': 'sum'
    })
    
    summary['capex_per_mw'] = summary[('capex_total', 'sum')] / summary[('capacity_mw', 'sum')].replace(0, 1)
    summary['opex_per_mwh'] = summary[('opex_total', 'sum')] / summary[('net_generation_mwh', 'sum')].replace(0, 1)
    
    summary.columns = [' '.join(col).strip() for col in summary.columns.values]
    summary = summary[[
        'capacity_mw sum', 
        'net_generation_mwh sum',
        'capex_total sum', 
        'capex_total mean',
        'capex_per_mw',
        'opex_total sum',
        'opex_total mean',
        'opex_per_mwh'
    ]]
    
    summary.columns = [
        'Capacidad Total (MW)',
        'Generación Total (MWh)',
        'CAPEX Total (USD)',
        'CAPEX Promedio (USD)',
        'CAPEX por MW (USD/MW)',
        'OPEX Total (USD)',
        'OPEX Promedio (USD)',
        'OPEX por MWh (USD/MWh)'
    ]
    
    explanation = """
    <div style="background-color:#f8f9fa; padding:15px; border-radius:5px; margin-bottom:20px">
        <h4>Tabla Resumen de Costos</h4>
        <p>Esta tabla presenta un resumen consolidado de los costos y métricas clave para los tipos de planta 
        y años seleccionados. Los valores incluyen:</p>
        <ul>
            <li><b>CAPEX Total/Promedio:</b> Inversión total en capital y promedio por planta</li>
            <li><b>CAPEX por MW:</b> Costo de capital por unidad de capacidad instalada</li>
            <li><b>OPEX Total/Promedio:</b> Costos operativos totales y promedio por planta</li>
            <li><b>OPEX por MWh:</b> Costo operativo por unidad de energía generada</li>
        </ul>
    </div>
    """
    
    display(HTML(explanation))
    
    styled_table = summary.style.format({
        'CAPEX Total (USD)': '${:,.0f}',
        'CAPEX Promedio (USD)': '${:,.0f}',
        'CAPEX por MW (USD/MW)': '${:,.0f}',
        'OPEX Total (USD)': '${:,.0f}',
        'OPEX Promedio (USD)': '${:,.0f}',
        'OPEX por MWh (USD/MWh)': '${:,.2f}',
        'Capacidad Total (MW)': '{:,.0f}',
        'Generación Total (MWh)': '{:,.0f}'
    }).background_gradient(cmap='Blues', subset=['CAPEX Total (USD)', 'OPEX Total (USD)'])
    
    styled_table = styled_table.set_properties(
        subset=['CAPEX por MW (USD/MW)', 'OPEX por MWh (USD/MWh)'],
        **{'background-color': '#f0f8ff', 'font-weight': 'bold'}
    )
    
    display(styled_table.set_caption("Resumen Consolidado de Costos"))

## Interfaz de Control Mejorada (MEJORADA)
def create_advanced_dashboard(df):
    df_clean = prepare_data(df)
    
    all_years = sorted(df_clean['report_year'].dt.year.unique()) if 'report_year' in df_clean.columns else []
    all_plant_types = sorted(df_clean['plant_type'].unique()) if 'plant_type' in df_clean.columns else []
    
    all_plant_names_base = sorted(df_clean['plant_name_ferc1_clean'].unique()) if 'plant_name_ferc1_clean' in df_clean.columns else sorted(df_clean['plant_name_ferc1'].unique())
    
    all_plant_names_with_type = sorted(df_clean['plant_name_with_type'].unique()) if 'plant_name_with_type' in df_clean.columns else ['No disponible']

    all_capacity_ranges = sorted(df_clean['capacity_mw_range'].unique()) if 'capacity_mw_range' in df_clean.columns else []
    all_capacity_ranges = [opt for opt in all_capacity_ranges if opt != 'Error de cálculo de rango']


    year_selector = widgets.SelectMultiple(
        options=['Todos'] + all_years,
        value=['Todos'],
        description='Años:',
        disabled=False,
        style={'description_width': 'initial'},
        layout={'width': '300px'}
    )
    
    plant_type_selector = widgets.SelectMultiple(
        options=['Todos'] + all_plant_types,
        value=['Todos'],
        description='Tipos de Planta:',
        disabled=False,
        style={'description_width': 'initial'},
        layout={'width': '300px'}
    )
    
    plant_name_search_bar = widgets.Text(
        value='',
        placeholder='Buscar nombre de planta...',
        description='Buscar:',
        disabled=False,
        style={'description_width': 'initial'},
        layout={'width': '300px'}
    )

    plant_name_selector = widgets.SelectMultiple(
        options=['Todos'] + all_plant_names_with_type, 
        value=['Todos'],
        description='Nombre de Planta:',
        disabled=False,
        style={'description_width': 'initial'},
        layout={'width': '300px'}
    )
    
    capacity_range_selector = widgets.SelectMultiple(
        options=['Todos'] + all_capacity_ranges,
        value=['Todos'],
        description='Rangos de Capacidad (MW):',
        disabled=False,
        style={'description_width': 'initial'},
        layout={'width': '300px'}
    )

    analysis_type = widgets.Dropdown(
        options=[
            ('Evolución de Costos', 'evolution'),
            ('Composición de Costos', 'composition'),
            ('Tabla Resumen', 'summary')
        ],
        value='evolution',
        description='Tipo de Análisis:',
        style={'description_width': 'initial'}
    )
    
    def get_selected_options(selected, all_options):
        if 'Todos' in selected or len(selected) == 0:
            return [opt for opt in all_options if opt not in ['No disponible', 'Error de cálculo de rango', 'nan']] 
        return [opt for opt in selected if opt not in ['No disponible', 'Error de cálculo de rango', 'nan']]

    def extract_plant_name_for_filter(plant_full_name_from_selector):
        if pd.isna(plant_full_name_from_selector) or plant_full_name_from_selector == 'No disponible':
            return 'No disponible' 
        
        name_without_type = plant_full_name_from_selector.split('(')[0].strip()
        
        name_cleaned = re.sub(r'\s*\(\d+\)\s*', '', name_without_type).strip()
        
        return name_cleaned
    
    def update_dependent_selectors(change):
        selected_years_filter = get_selected_options(year_selector.value, all_years)
        
        selected_years_dt_filter = [pd.to_datetime(str(year)) for year in selected_years_filter]
        
        filtered_for_dependencies_df = df_clean[df_clean['report_year'].isin(selected_years_dt_filter)] if selected_years_dt_filter else df_clean
        
        current_plant_type_options = sorted(filtered_for_dependencies_df['plant_type'].unique()) if 'plant_type' in filtered_for_dependencies_df.columns else []
        new_plant_type_options = ['Todos'] + [opt for opt in current_plant_type_options if opt != 'nan'] 
        current_selection_types = [pt for pt in plant_type_selector.value if pt in new_plant_type_options]
        plant_type_selector.options = new_plant_type_options
        plant_type_selector.value = current_selection_types if current_selection_types else ['Todos']

        current_capacity_range_options = sorted(filtered_for_dependencies_df['capacity_mw_range'].unique()) if 'capacity_mw_range' in filtered_for_dependencies_df.columns else []
        current_capacity_range_options = [opt for opt in current_capacity_range_options if opt not in ['Error de cálculo de rango', 'nan']]
        new_capacity_range_options = ['Todos'] + current_capacity_range_options
        current_selection_ranges = [cr for cr in capacity_range_selector.value if cr in new_capacity_range_options]
        capacity_range_selector.options = new_capacity_range_options
        capacity_range_selector.value = current_selection_ranges if current_selection_ranges else ['Todos']
        
        update_plant_name_options(None)

    def update_plant_name_options(change):
        search_term = plant_name_search_bar.value.lower()
        
        selected_years_filter = get_selected_options(year_selector.value, all_years)
        selected_plant_types_filter = get_selected_options(plant_type_selector.value, all_plant_types)
        selected_capacity_ranges_filter = get_selected_options(capacity_range_selector.value, all_capacity_ranges) 

        selected_years_dt_filter = [pd.to_datetime(str(year)) for year in selected_years_filter]

        filtered_df_for_plants = df_clean[
            (df_clean['report_year'].isin(selected_years_dt_filter) if 'report_year' in df_clean.columns else True) & 
            (df_clean['plant_type'].isin(selected_plant_types_filter) if 'plant_type' in df_clean.columns else True) &
            (df_clean['capacity_mw_range'].isin(selected_capacity_ranges_filter) if 'capacity_mw_range' in df_clean.columns else True) 
        ]

        if 'plant_name_with_type' in filtered_df_for_plants.columns:
            current_relevant_plant_names = sorted(filtered_df_for_plants['plant_name_with_type'].unique())
            
            filtered_by_search_plant_names = [
                pn for pn in current_relevant_plant_names
                if search_term in pn.lower()
            ]
            
            new_plant_name_options = ['Todos'] + filtered_by_search_plant_names
            
            current_selection_plants = [p for p in plant_name_selector.value if p in new_plant_name_options]
            plant_name_selector.options = new_plant_name_options
            plant_name_selector.value = current_selection_plants if current_selection_plants else ['Todos']
        else:
            plant_name_selector.options = ['No disponible']
            plant_name_selector.value = ['No disponible']
            
    year_selector.observe(update_dependent_selectors, names='value')
    plant_type_selector.observe(update_dependent_selectors, names='value')
    capacity_range_selector.observe(update_dependent_selectors, names='value') 
    plant_name_search_bar.observe(update_plant_name_options, names='value')

    update_dependent_selectors(None) 

    def update_analysis(analysis_type_val, selected_years_val, selected_plant_types_val, selected_plants_val, selected_capacity_ranges_val):
        
        years_to_use = get_selected_options(selected_years_val, all_years)
        plant_types_to_use = get_selected_options(selected_plant_types_val, all_plant_types)
        capacity_ranges_to_use = get_selected_options(selected_capacity_ranges_val, all_capacity_ranges) 
        
        selected_plants_from_selector_with_type = get_selected_options(selected_plants_val, plant_name_selector.options)
        
        plants_to_use_for_filter = [extract_plant_name_for_filter(p) for p in selected_plants_from_selector_with_type]

        plants_to_use_for_filter = list(set(plants_to_use_for_filter))

        if 'Todos' in plants_to_use_for_filter or not plants_to_use_for_filter:
            plants_to_use_for_filter = all_plant_names_base
        
        if analysis_type_val == 'evolution':
            plot_cost_evolution(df_clean, years_to_use, plant_types_to_use, plants_to_use_for_filter, capacity_ranges_to_use)
        elif analysis_type_val == 'composition':
            plot_cost_composition(df_clean, years_to_use, plant_types_to_use, plants_to_use_for_filter, capacity_ranges_to_use)
        elif analysis_type_val == 'summary':
            display_summary_table(df_clean, years_to_use, plant_types_to_use, plants_to_use_for_filter, capacity_ranges_to_use)
    
    ui = widgets.VBox([
        widgets.HBox([year_selector, plant_type_selector]),
        widgets.HBox([plant_name_search_bar, plant_name_selector]),
        widgets.HBox([capacity_range_selector]), 
        analysis_type
    ])
    
    out = widgets.interactive_output(
        update_analysis,
        {
            'analysis_type_val': analysis_type,
            'selected_years_val': year_selector,
            'selected_plant_types_val': plant_type_selector,
            'selected_plants_val': plant_name_selector, 
            'selected_capacity_ranges_val': capacity_range_selector 
        }
    )
    
    display(ui, out)


# Assuming df2 is already loaded in your environment, you can run the dashboard for df2
create_advanced_dashboard(df2)

Output()

In [7]:
# --- EJEMPLO 1: Análisis para el año 2024 ---
print("--- Análisis de Costos de Generación para el Año 2024 ---")
perform_descriptive_analysis(df, years=[2023])

--- Análisis de Costos de Generación para el Año 2024 ---
